In [1]:
import numpy as np
from torch.utils.data import Dataset,DataLoader
import csv
import torch
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2


# 开始读入数据。


In [2]:
with open(r'C:\Users\10056\Desktop\科研\data_set\ml2022spring-hw1\covid.test.csv', 'r') as f:
    train_data = f.readlines()
 
    train_data = [line.split('\n') for line in train_data][1:]  #分行之后不要第一行
    train_data = [each[0].split(',') for each in train_data]    #对于每一行 去掉后面的空格
    print(len(train_data[0]))
    train_data = np.array(train_data)         #转换成numpy的矩阵
 
    train_x = train_data[:,1:-1]     # x是数据，y是标签 。第一个冒号表示所有行，第二个冒号表示
    train_y = train_data[:,-1]      #列。所以x就是第2列到倒数第二列。y就是倒数第一列。

117


In [3]:
train_data

array([['0', '0', '0', ..., '12.4525703', '9.4000244', '30.5508734'],
       ['1', '0', '0', ..., '11.7475281', '7.4386058', '35.3648714'],
       ['2', '0', '0', ..., '12.8980889', '10.2147107', '37.0287053'],
       ...,
       ['1075', '0', '0', ..., '13.6469982', '10.9437572', '32.5646431'],
       ['1076', '0', '0', ..., '15.7997773', '11.4043682', '43.8621766'],
       ['1077', '0', '0', ..., '14.1463771', '10.8294526', '33.4778761']],
      dtype='<U10')

In [4]:
train_x


array([['0', '0', '0', ..., '3.4721051', '12.4525703', '9.4000244'],
       ['0', '0', '0', ..., '5.9923323', '11.7475281', '7.4386058'],
       ['0', '0', '0', ..., '13.9516313', '12.8980889', '10.2147107'],
       ...,
       ['0', '0', '0', ..., '3.207936', '13.6469982', '10.9437572'],
       ['0', '0', '0', ..., '2.9658171', '15.7997773', '11.4043682'],
       ['0', '0', '0', ..., '2.6319415', '14.1463771', '10.8294526']],
      dtype='<U10')

In [5]:
train_y

array(['30.5508734', '35.3648714', '37.0287053', ..., '32.5646431',
       '43.8621766', '33.4778761'], dtype='<U10')

In [6]:
 
def get_feature_importance(feature_data, label_data, k =4,column = None):
    """
    此处省略 feature_data, label_data 的生成代码。
    如果是 CSV 文件，可通过 read_csv() 函数获得特征和标签。
    """
    model = SelectKBest(chi2, k=k)#选择k个最佳特征
    X_new = model.fit_transform(feature_data, label_data)
    #feature_data是特征数据，label_data是标签数据，该函数可以选择出k个特征
    print('x_new', X_new)
    scores = model.scores_
    # 按重要性排序，选出最重要的 k 个
    indices = np.argsort(scores)[::-1] #找到重要K个的下标
    if column:
        k_best_features = [column[i+1] for i in indices[0:k].tolist()]
        print('k best features are: ',k_best_features)
    return X_new, indices[0:k]

In [7]:
import numpy as np
from torch.utils.data import Dataset,DataLoader
import csv
import torch
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
 
def get_feature_importance(feature_data, label_data, k =4,column = None):
    """
    此处省略 feature_data, label_data 的生成代码。
    如果是 CSV 文件，可通过 read_csv() 函数获得特征和标签。
    """
    model = SelectKBest(chi2, k=k)#选择k个最佳特征
    X_new = model.fit_transform(feature_data, label_data)
    #feature_data是特征数据，label_data是标签数据，该函数可以选择出k个特征
    print('x_new', X_new)
    scores = model.scores_
    # 按重要性排序，选出最重要的 k 个
    indices = np.argsort(scores)[::-1] #找到重要K个的下标
    if column:
        k_best_features = [column[i+1] for i in indices[0:k].tolist()]
        print('k best features are: ',k_best_features)
    return X_new, indices[0:k]
 
 
class covidDataset(Dataset):
    def __init__(self, path, mode, feature_dim):
        with open(path,'r') as f:
            csv_data = list(csv.reader(f))
            column = csv_data[0]
            train_x = np.array(csv_data)[1:][:,1:-1]
            train_y = np.array(csv_data)[1:][:,-1]
            _,col_indices = get_feature_importance(train_x,train_y,feature_dim,column)
            col_indices = col_indices.tolist()   #得到重要列的下标
            csv_data = np.array(csv_data[1:])[:,1:].astype(float)
            if mode == 'train':       #如果读的是训练数据 就逢5取4  indices是数据下标
                indices = [i for i in range(len(csv_data)) if i % 5 != 0]
                self.y = torch.LongTensor(csv_data[indices,-1])
            elif mode == 'val':  #如果读的是验证数据 就逢5取1  indices是数据下标
                indices = [i for i in range(len(csv_data)) if i % 5 == 0]
                # data = torch.tensor(csv_data[indices,col_indices])
                self.y = torch.LongTensor(csv_data[indices,-1])
            else:      #如果读的是测试数据 就全取了
                indices = [i for i in range(len(csv_data))]
            data = torch.tensor(csv_data[indices,:]) #取行
            self.data = data[:,col_indices]   #取列
            self.mode = mode
            self.data = (self.data - self.data.mean(dim=0,keepdim=True)) /self.data.std(dim=0,keepdim=True)     #这里将数据归一化。
            assert feature_dim == self.data.shape[1]
 
 
            print('Finished reading the {} set of COVID19 Dataset ({} samples found, each dim = {})'
                  .format(mode, len(self.data), feature_dim))
 
    def __getitem__(self, item):
        if self.mode == 'test':
            return self.data[item].float()
        else :
            return self.data[item].float(), self.y[item]
    def __len__(self):
        return len(self.data)

In [8]:
import torch.nn as nn
 
class myNet(nn.Module):
    def __init__(self,inDim):
        super(myNet,self).__init__()
        self.fc1 = nn.Linear(inDim, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64,1)
 
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        if len(x.size()) > 1:
            return x.squeeze(1)     #如果批量大小不为1 这里才需要展平。 
        else:
            return x

In [9]:
   for i in range(epoch):
        start_time = time.time()
        model.train()   #开启训练
        train_loss = 0.0
        val_loss = 0.0
        for data in trainloader:
            optimizer.zero_grad()
            x , target = data[0].to(device), data[1].to(torch.float32).to(device)  
            #从loader里取一批数据
            pred = model(x)  #经过模型预测
            bat_loss = loss(pred, target, model)  #计算loss
            bat_loss.backward()     #梯度回传
            optimizer.step()      #计算
            train_loss += bat_loss.detach().cpu().item()    #记录loss值 注意要从gpu上取下来
#再从张量里取出来
 
        plt_train_loss . append(train_loss/trainset.__len__())  #记录

NameError: name 'epoch' is not defined